In [ ]:
! pip install tensorflow


In [5]:
! pip install pandas

Defaulting to user installation because normal site-packages is not writeable
  Using cached pandas-2.2.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.1 MB)
  Using cached tzdata-2024.2-py2.py3-none-any.whl (346 kB)
You should consider upgrading via the '/sw/eb/sw/Python/3.10.4-GCCcore-11.3.0/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
! pip install scikit-learn

In [ ]:
! pip install opencv-python

In [5]:
# preprocessing/data_preprocessing.py

import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler
import joblib

def preprocess_meteorological_data(file_path):
    # Load data
    met_data = pd.read_csv(file_path)

    # Combine DATE and MST into a single datetime column
    # met_data['Timestamp'] = pd.to_datetime(met_data['DATE'] + ' ' + met_data['MST'])
    # Combine DATE and MST into a single datetime column
    met_data['Timestamp'] = pd.to_datetime(met_data['datetime'].astype(str), format='%Y%m%d%H%M%S')

    # Sort data by Timestamp
    met_data.sort_values('Timestamp', inplace=True)

    # Handle missing values in input features using KNN imputation
    input_features = [
        'Tower Dry Bulb Temp [deg C]', 'Tower RH [%]', 'Station Pressure [mBar]',
        'Avg Wind Speed @ 6ft [m/s]', 'Avg Wind Direction @ 6ft [deg from N]'
    ]

    # Initialize KNN imputer
    imputer = KNNImputer(n_neighbors=5)

    # Fit and transform the input features
    met_data_imputed = imputer.fit_transform(met_data[input_features])

    # Update the DataFrame with imputed values
    met_data[input_features] = met_data_imputed

    # Handle missing values in the target variable separately
    target_variable = 'Global CMP22 (vent/cor) [W/m^2]'

    # Optionally interpolate missing target values
    # met_data[target_variable].interpolate(method='time', inplace=True)
    
    # Option 2: Drop rows with missing target values (uncomment if preferred)
    met_data.dropna(subset=[target_variable], inplace=True)

    # Rename columns for simplicity
    met_data.rename(columns={
        'Tower Dry Bulb Temp [deg C]': 'Temperature',
        'Tower RH [%]': 'Humidity',
        'Station Pressure [mBar]': 'Pressure',
        'Avg Wind Speed @ 6ft [m/s]': 'Wind Speed',
        'Avg Wind Direction @ 6ft [deg from N]': 'Wind Direction',
        'Global CMP22 (vent/cor) [W/m^2]': 'Irradiance'
    }, inplace=True)

    # Feature scaling for input features
    scaler = MinMaxScaler()
    met_data[['Temperature', 'Humidity', 'Pressure', 'Wind Speed']] = scaler.fit_transform(
        met_data[['Temperature', 'Humidity', 'Pressure', 'Wind Speed']]
    )
    joblib.dump(scaler, 'scaler_y.pkl')

    # Wind Direction encoding (convert degrees to sine and cosine components)
    met_data['Wind Dir Sin'] = np.sin(np.deg2rad(met_data['Wind Direction']))
    met_data['Wind Dir Cos'] = np.cos(np.deg2rad(met_data['Wind Direction']))
    met_data.drop('Wind Direction', axis=1, inplace=True)

    # Temporal features
    met_data['Hour'] = met_data['Timestamp'].dt.hour / 23.0  # Normalize Hour
    met_data['DayOfYear'] = met_data['Timestamp'].dt.dayofyear / 365.0  # Normalize DayOfYear

    # Prepare target variables (future irradiance)
    target = 'Irradiance'
    for minutes in [5, 15, 30, 60]:
        met_data[f'Irradiance_{minutes}min_ahead'] = met_data[target].shift(-minutes)

    # **Removed the line that drops rows with NaN values after shifting**
    # We will handle dropping NaN values after merging with images
    # met_data.dropna(inplace=True)

    # Reset index and return the processed DataFrame
    return met_data.reset_index(drop=True)

In [ ]:
# preprocessing/image_preprocessing.py

import cv2
import numpy as np
import glob
import os
import pandas as pd
from tqdm import tqdm

def preprocess_images(image_folder):
    image_paths = sorted(glob.glob(os.path.join(image_folder, '*.jpg')))
    images = []
    image_timestamps = []

    for path in tqdm(image_paths, desc="Processing images"):
        # Extract timestamp from image filename
        # Assuming filename format: YYYYMMDDHHMMSS.jpg
        filename = os.path.basename(path)
        timestamp_str = filename.replace('.jpg', '')
        timestamp = pd.to_datetime(timestamp_str, format='%Y%m%d%H%M%S')

        img = cv2.imread(path)
        if img is None:
            continue  # Skip if the image is not readable
        # Iterates over each image path, reads the image using OpenCV, and resizes it to 128x128 pixels
        # Normalizes pixel values to the range [0, 1] by dividing by 255
        img = cv2.resize(img, (128, 128))
        img = img / 255.0  # Normalize pixel values
        images.append(img)
        image_timestamps.append(timestamp)

    return images, image_timestamps

In [ ]:
met_data = preprocess_meteorological_data('data/combined_data.csv')
met_data


,datetime,Irradiance,Temperature,Humidity,Wind Speed,Pressure,Timestamp,Wind Dir Sin,Wind Dir Cos,Hour,DayOfYear,Irradiance_5min_ahead,Irradiance_15min_ahead,Irradiance_30min_ahead,Irradiance_60min_ahead
0,20230101065000,-0.847312,0.577610,0.536853,0.110047,0.184172,2023-01-01 06:50:00,-0.987688,-0.156434,0.26087,0.002740,-0.718261,-0.419715,1.97398,26.0810
1,20230101065100,-0.827263,0.577795,0.540062,0.125623,0.184632,2023-01-01 06:51:00,-0.999976,0.006981,0.26087,0.002740,-0.708250,-0.333484,2.17521,27.4360
2,20230101065200,-0.821106,0.577637,0.542165,0.124611,0.185540,2023-01-01 06:52:00,-0.998027,-0.062791,0.26087,0.002740,-0.662647,-0.257296,2.46600,28.8680
3,20230101065300,-0.789932,0.577663,0.544046,0.112928,0.185629,2023-01-01 06:53:00,-0.999328,0.036644,0.26087,0.002740,-0.594409,-0.166430,2.73742,30.3134
4,20230101065400,-0.734090,0.577346,0.545374,0.123676,0.185092,2023-01-01 06:54:00,-0.999328,0.036644,0.26087,0.002740,-0.542234,-0.025133,3.04919,31.7266
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19458,20230130174600,-0.444423,0.154904,0.800797,0.139252,0.220520,2023-01-30 17:46:00,0.956509,0.291704,0.73913,0.082192,NaN,NaN,NaN,NaN
19459,20230130174700,-0.432117,0.154639,0.801903,0.134346,0.220687,2023-01-30 17:47:00,0.811880,0.583825,0.73913,0.082192,NaN,NaN,NaN,NaN
19460,20230130174800,-0.451800,0.154639,0.800797,0.113941,0.221415,2023-01-30 17:48:00,0.956356,0.292205,0.73913,0.082192,NaN,NaN,NaN,NaN
19461,20230130174900,-0.482391,0.154110,0.801903,0.113941,0.222016,2023-01-30 17:49:00,0.924280,0.381716,0.73913,0.082192,NaN,NaN,NaN,NaN


In [ ]:
images, image_timestamps = preprocess_images('/data/training_images/')
# image_timestamps

In [35]:
# training/train_model.py

import numpy as np
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from models.hybrid import create_hybrid_model
import pandas as pd

def train_hybrid_model(met_data, images, image_timestamps):
    # Prepare data
    sequence_length = 60  # Number of past minutes to consider
    features = ['Temperature', 'Humidity', 'Pressure', 'Wind Speed', 'Wind Dir Sin', 'Wind Dir Cos',
                'Hour', 'DayOfYear']

    # Align images with meteorological data
    merged_data = align_data_with_images(met_data, images, image_timestamps)

    # Extract features and targets
    X_num = merged_data[features].values
    y = merged_data[[f'Irradiance_{minutes}min_ahead' for minutes in [5, 15, 30, 60]]].values
    X_img = np.array(merged_data['Image'].tolist())

    # Create sequences
    def create_sequences(X_num, X_img, y, seq_length):
        X_num_seq, X_img_seq, y_seq = [], [], []
        for i in range(len(X_num) - seq_length):
            X_num_seq.append(X_num[i:i+seq_length])
            X_img_seq.append(X_img[i+seq_length-1])  # Use image at last timestamp
            y_seq.append(y[i+seq_length-1])
        return np.array(X_num_seq), np.array(X_img_seq), np.array(y_seq)

    X_num_seq, X_img_seq, y_seq = create_sequences(X_num, X_img, y, sequence_length)

    # Train-test split
    split_index = int(0.8 * len(X_num_seq))
    X_num_train, X_num_test = X_num_seq[:split_index], X_num_seq[split_index:]
    X_img_train, X_img_test = X_img_seq[:split_index], X_img_seq[split_index:]
    y_train, y_test = y_seq[:split_index], y_seq[split_index:]

    # Create model
    num_features = X_num_train.shape[2]
    model = create_hybrid_model(sequence_length, num_features)

    # Compile model
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])

    # Callbacks
    early_stopping = EarlyStopping(patience=5, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(factor=0.5, patience=3)

    # Train model
    history = model.fit(
        [X_img_train, X_num_train],
        y_train,
        validation_data=([X_img_test, X_num_test], y_test),
        epochs=50,
        batch_size=32,
        callbacks=[early_stopping, reduce_lr]
    )

    return model, history, merged_data

def align_data_with_images(met_data, images, image_timestamps):
    # Create a DataFrame for image timestamps
    image_df = pd.DataFrame({'Timestamp': image_timestamps, 'Image': images})
    image_df['Timestamp'] = pd.to_datetime(image_df['Timestamp'])

    # Merge meteorological data with images using an inner join
    met_data['Timestamp'] = pd.to_datetime(met_data['Timestamp'])
    merged_data = pd.merge(met_data, image_df, on='Timestamp', how='inner')

    # Prepare target variables (future irradiance)
    target = 'Irradiance'
    for minutes in [5, 15, 30, 60]:
        merged_data[f'Irradiance_{minutes}min_ahead'] = merged_data[target].shift(-minutes)

    # Drop rows with any remaining missing values (after shifting)
    merged_data.dropna(inplace=True)

    # Reset index
    merged_data.reset_index(drop=True, inplace=True)

    return merged_data

In [ ]:
# Train the model
model, history, merged_data = train_hybrid_model(met_data, images, image_timestamps)

# Save the model
model.save('trainedModels/model.keras')

Epoch 1/50


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/models/functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['Image_Input', 'LSTM_Input']. Received: the structure of inputs=('*', '*')
  warnings.warn(


44/44 ━━━━━━━━━━━━━━━━━━━━ 10s 158ms/step - loss: 51387.0742 - mae: 177.9589 - val_loss: 34245.9961 - val_mae: 153.7390 - learning_rate: 0.0010
Epoch 2/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 6s 146ms/step - loss: 29531.3281 - mae: 143.8452 - val_loss: 31420.1367 - val_mae: 149.2367 - learning_rate: 0.0010
Epoch 3/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 7s 164ms/step - loss: 26398.3242 - mae: 136.4072 - val_loss: 22572.0078 - val_mae: 123.4668 - learning_rate: 0.0010
Epoch 4/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 7s 149ms/step - loss: 17380.7734 - mae: 103.8256 - val_loss: 20566.5664 - val_mae: 114.3433 - learning_rate: 0.0010
Epoch 5/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 7s 153ms/step - loss: 15953.9658 - mae: 98.9433 - val_loss: 16924.3262 - val_mae: 103.2513 - learning_rate: 0.0010
Epoch 6/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 8s 175ms/step - loss: 14833.2129 - mae: 92.6789 - val_loss: 20342.1211 - val_mae: 111.1444 - learning_rate: 0.0010
Epoch 7/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 7s 159ms/step - loss: 14472.0195 - mae: 91.6104 - 

In [55]:
# evaluation/evaluate_model.py

import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
import os
from datetime import datetime

def evaluate_model(model, merged_data):
    """
    Evaluates the regression model on the provided merged_data.
    Saves evaluation plots in a uniquely named subfolder within 'evaluation_plots'.
    
    Parameters:
    - model: Trained Keras model.
    - merged_data: pandas DataFrame containing meteorological data and associated images.
    """
    
    # -----------------------------
    # 1. Setup Directory for Saving Plots
    # -----------------------------
    
    # Define the base directory for evaluation plots
    base_dir = 'evaluation_plots'
    
    # Create the base directory if it doesn't exist
    if not os.path.exists(base_dir):
        os.makedirs(base_dir)
        print(f"Created base directory for evaluation plots at '{base_dir}'.")
    
    # Generate a unique subfolder name using the current timestamp
    run_timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    run_folder = os.path.join(base_dir, f'run_{run_timestamp}')
    
    # Create the subfolder
    os.makedirs(run_folder, exist_ok=True)
    print(f"Created run-specific directory at '{run_folder}'.")
    
    # -----------------------------
    # 2. Prepare Data for Evaluation
    # -----------------------------
    
    # Define the sequence length and feature columns
    sequence_length = 60
    features = [
        'Temperature', 'Humidity', 'Pressure', 'Wind Speed',
        'Wind Dir Sin', 'Wind Dir Cos', 'Hour', 'DayOfYear'
    ]  # Exclude direct current irradiance as a feature

    # Extract feature values and target variables
    X_num = merged_data[features].values
    y = merged_data[[f'Irradiance_{minutes}min_ahead' for minutes in [5, 15, 30, 60]]].values
    X_img = np.array(merged_data['Image'].tolist())

    # -----------------------------
    # 3. Create Sequences
    # -----------------------------
    
    def create_sequences(X_num, X_img, y, seq_length):
        """
        Creates input sequences for the model.
        
        Parameters:
        - X_num: Numpy array of numerical features.
        - X_img: Numpy array of images.
        - y: Numpy array of target variables.
        - seq_length: Length of the input sequences.
        
        Returns:
        - Tuple of Numpy arrays: (X_num_seq, X_img_seq, y_seq)
        """
        X_num_seq, X_img_seq, y_seq = [], [], []
        for i in range(len(X_num) - seq_length):
            X_num_seq.append(X_num[i:i+seq_length])
            X_img_seq.append(X_img[i+seq_length-1])  # Use image at the last timestamp
            y_seq.append(y[i+seq_length-1])
        return np.array(X_num_seq), np.array(X_img_seq), np.array(y_seq)

    # Generate sequences
    X_num_seq, X_img_seq, y_seq = create_sequences(X_num, X_img, y, sequence_length)
    print(f"Created {len(X_num_seq)} sequences for evaluation.")

    # -----------------------------
    # 4. Split Data into Test Set
    # -----------------------------
    
    # Define the split index for the last 20% as the test set
    split_index = int(0.8 * len(X_num_seq))
    
    # Split the data
    X_num_test = X_num_seq[split_index:]
    X_img_test = X_img_seq[split_index:]
    y_test = y_seq[split_index:]
    
    print(f"Evaluation split: {len(X_num_test)} samples.")

    # -----------------------------
    # 5. Make Predictions
    # -----------------------------
    
    # Generate predictions using the trained model
    y_pred = model.predict([X_img_test, X_num_test])
    print("Generated predictions for the test set.")

    # -----------------------------
    # 6. Calculate and Save Metrics and Plots
    # -----------------------------
    
    horizons = [5, 15, 30, 60]  # Prediction horizons in minutes
    
    # Initialize a text file to save metrics
    metrics_file = os.path.join(run_folder, 'metrics.txt')
    with open(metrics_file, 'w') as f:
        f.write("Evaluation Metrics:\n")
        f.write("===================\n\n")
    
    for i, minutes in enumerate(horizons):
        # Calculate metrics for each horizon
        rmse = np.sqrt(mean_squared_error(y_test[:, i], y_pred[:, i]))
        mae = mean_absolute_error(y_test[:, i], y_pred[:, i])
        r2 = r2_score(y_test[:, i], y_pred[:, i])
        metric_str = f"{minutes}-Minute Ahead Prediction - RMSE: {rmse:.2f}, MAE: {mae:.2f}, R²: {r2:.2f}"
        print(metric_str)
        
        # Append metrics to the text file
        with open(metrics_file, 'a') as f:
            f.write(metric_str + "\n")
        
        # Plot Actual vs Predicted
        plt.figure(figsize=(10, 4))
        plt.plot(y_test[:, i], label='Actual', alpha=0.7)
        plt.plot(y_pred[:, i], label='Predicted', alpha=0.7)
        plt.title(f'{minutes}-Minute Ahead Prediction')
        plt.xlabel('Samples')
        plt.ylabel('Irradiance (W/m²)')
        plt.legend()
        plt.tight_layout()
        
        # Save the plot
        plot_filename = f'{minutes}_min_ahead_prediction.png'
        plot_path = os.path.join(run_folder, plot_filename)
        plt.savefig(plot_path)
        plt.close()
        print(f"Saved plot: {plot_path}")
        
        # Plot Scatter of Actual vs Predicted
        plt.figure(figsize=(6, 6))
        plt.scatter(y_test[:, i], y_pred[:, i], alpha=0.5)
        plt.plot([y_test[:, i].min(), y_test[:, i].max()],
                 [y_test[:, i].min(), y_test[:, i].max()],
                 'r--', lw=2)
        plt.title(f'Actual vs Predicted Irradiance ({minutes} min Ahead)')
        plt.xlabel('Actual Irradiance (W/m²)')
        plt.ylabel('Predicted Irradiance (W/m²)')
        plt.tight_layout()
        
        # Save the scatter plot
        scatter_filename = f'actual_vs_predicted_{minutes}_min_ahead.png'
        scatter_path = os.path.join(run_folder, scatter_filename)
        plt.savefig(scatter_path)
        plt.close()
        print(f"Saved scatter plot: {scatter_path}")
    
    # -----------------------------
    # 7. Calculate and Save Overall Metrics and Plots
    # -----------------------------
    
    # Calculate overall performance metrics across all horizons
    overall_rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    overall_mae = mean_absolute_error(y_test, y_pred)
    overall_r2 = r2_score(y_test, y_pred)
    overall_metric_str = f"Overall Performance - RMSE: {overall_rmse:.2f}, MAE: {overall_mae:.2f}, R²: {overall_r2:.2f}"
    print(overall_metric_str)
    
    # Append overall metrics to the text file
    with open(metrics_file, 'a') as f:
        f.write("\n" + overall_metric_str + "\n")
    
    # Plot Overall Actual vs Predicted
    plt.figure(figsize=(6,6))
    plt.scatter(y_test, y_pred, alpha=0.5)
    plt.plot([y_test.min(), y_test.max()],
             [y_test.min(), y_test.max()],
             'r--', lw=2)
    plt.title('Actual vs Predicted Irradiance (Overall)')
    plt.xlabel('Actual Irradiance (W/m²)')
    plt.ylabel('Predicted Irradiance (W/m²)')
    plt.tight_layout()
    
    # Save the overall scatter plot
    overall_scatter_filename = 'actual_vs_predicted_overall.png'
    overall_scatter_path = os.path.join(run_folder, overall_scatter_filename)
    plt.savefig(overall_scatter_path)
    plt.close()
    print(f"Saved overall scatter plot: {overall_scatter_path}")

In [56]:
evaluate_model(model, merged_data)

Created base directory for evaluation plots at 'evaluation_plots'.
Created run-specific directory at 'evaluation_plots/run_20241118_213501'.
Created 1759 sequences for evaluation.
Evaluation split: 352 samples.
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step
Generated predictions for the test set.
5-Minute Ahead Prediction - RMSE: 96.40, MAE: 73.15, R²: 0.73
Saved plot: evaluation_plots/run_20241118_213501/5_min_ahead_prediction.png
Saved scatter plot: evaluation_plots/run_20241118_213501/actual_vs_predicted_5_min_ahead.png
15-Minute Ahead Prediction - RMSE: 122.37, MAE: 96.39, R²: 0.57
Saved plot: evaluation_plots/run_20241118_213501/15_min_ahead_prediction.png
Saved scatter plot: evaluation_plots/run_20241118_213501/actual_vs_predicted_15_min_ahead.png
30-Minute Ahead Prediction - RMSE: 125.33, MAE: 103.67, R²: 0.55
Saved plot: evaluation_plots/run_20241118_213501/30_min_ahead_prediction.png
Saved scatter plot: evaluation_plots/run_20241118_213501/actual_vs_predicted_30_min_ahead.png
60-Minu